In [66]:
import tensorflow as tf
import numpy as np
from tfkan.layers import Conv1DKAN
from tfkan.layers import DenseKAN
from sklearn.model_selection import train_test_split

In [55]:
def generate_sine_wave(seq_length, num_samples, noise=0.1):
    """
    generate a sin(t) wave time sequence with gaussian noise,\
    the target y is the next step of the sequence. 
    """
    X = np.zeros((num_samples, seq_length, 1))
    y = np.zeros((num_samples, 1))
    
    for i in range(num_samples):
        start = np.random.rand() * 2 * np.pi
        grids = np.linspace(start, start + (seq_length + 1) * 0.1, seq_length + 1)
        X[i, :, 0] = np.sin(grids[:-1]) + noise * np.random.randn(seq_length)
        # predict the next step
        y[i, 0] = np.sin(grids[-1])
    
    return X, y

In [67]:
X, y = generate_sine_wave(seq_length=32, num_samples=1000)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [69]:
model = tf.keras.Sequential([
    Conv1DKAN(3, kernel_size=6, strides=1, kan_kwargs={"grid_size": 3}),
    Conv1DKAN(1, kernel_size=6, strides=2, kan_kwargs={"grid_size": 3}),
    tf.keras.layers.Flatten(),
    DenseKAN(1, grid_size=3)
])
model.build(input_shape=(None, 32, 1))
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1dkan_20 (Conv1DKAN)    (None, 27, 3)             189       
                                                                 
 conv1dkan_21 (Conv1DKAN)    (None, 11, 1)             307       
                                                                 
 flatten_6 (Flatten)         (None, 11)                0         
                                                                 
 dense_kan_14 (DenseKAN)     (None, 1)                 188       
                                                                 
Total params: 684 (2.67 KB)
Trainable params: 334 (1.30 KB)
Non-trainable params: 350 (1.37 KB)
_________________________________________________________________


In [70]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss='mse',
    metrics=['mae']
)

In [71]:
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/50
22/22 [==============================] - 1s 18ms/step - loss: 0.4547 - mae: 0.5968 - val_loss: 0.4162 - val_mae: 0.5614
Epoch 2/50
22/22 [==============================] - 0s 5ms/step - loss: 0.3648 - mae: 0.5330 - val_loss: 0.3186 - val_mae: 0.4903
Epoch 3/50
22/22 [==============================] - 0s 6ms/step - loss: 0.2687 - mae: 0.4568 - val_loss: 0.2119 - val_mae: 0.3986
Epoch 4/50
22/22 [==============================] - 0s 5ms/step - loss: 0.1667 - mae: 0.3577 - val_loss: 0.1143 - val_mae: 0.2892
Epoch 5/50
22/22 [==============================] - 0s 8ms/step - loss: 0.0812 - mae: 0.2398 - val_loss: 0.0472 - val_mae: 0.1747
Epoch 6/50
22/22 [==============================] - 0s 8ms/step - loss: 0.0359 - mae: 0.1543 - val_loss: 0.0241 - val_mae: 0.1243
Epoch 7/50
22/22 [==============================] - 0s 8ms/step - loss: 0.0221 - mae: 0.1216 - val_loss: 0.0172 - val_mae: 0.1055
Epoch 8/50
22/22 [==============================] - 0s 8ms/step - loss: 0.0165 - mae: 0.1